## Chat Assistant for SQLite Database

In [2]:
import sqlite3
from datetime import datetime

def create_database():
    """Create the SQLite database with sample data"""
    conn = sqlite3.connect('company.db')
    cursor = conn.cursor()

    # Create Employees table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Employees (
        ID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Department TEXT NOT NULL,
        Salary INTEGER NOT NULL,
        Hire_Date DATE NOT NULL
    )
    ''')

    # Create Departments table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Departments (
        ID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Manager TEXT NOT NULL
    )
    ''')

    # Insert sample data into Employees
    employees_data = [
        (1, 'Alice', 'Sales', 50000, '2021-01-15'),
        (2, 'Bob', 'Engineering', 70000, '2020-06-10'),
        (3, 'Charlie', 'Marketing', 60000, '2022-03-20')
    ]

    # Insert sample data into Departments
    departments_data = [
        (1, 'Sales', 'Alice'),
        (2, 'Engineering', 'Bob'),
        (3, 'Marketing', 'Charlie')
    ]

    # Insert data
    cursor.executemany('INSERT OR REPLACE INTO Employees VALUES (?,?,?,?,?)', employees_data)
    cursor.executemany('INSERT OR REPLACE INTO Departments VALUES (?,?,?)', departments_data)

    conn.commit()
    conn.close()

if __name__ == "__main__":
    create_database()

In [ ]:
import sqlite3
import re
from flask import Flask, request, jsonify, render_template
import threading
import os

app = Flask(__name__)

# Ensure the database is in the same directory as the script
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATABASE_PATH = os.path.join(BASE_DIR, 'company.db')

def create_database():
    """Create the SQLite database with sample data if it doesn't exist"""
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()

    # Create Employees table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Employees (
        ID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Department TEXT NOT NULL,
        Salary INTEGER NOT NULL,
        Hire_Date DATE NOT NULL
    )
    ''')

    # Create Departments table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Departments (
        ID INTEGER PRIMARY KEY,
        Name TEXT NOT NULL,
        Manager TEXT NOT NULL
    )
    ''')

    # Check if data exists before inserting
    cursor.execute('SELECT COUNT(*) FROM Employees')
    if cursor.fetchone()[0] == 0:
        # Insert sample data into Employees
        employees_data = [
            (1, 'Alice', 'Sales', 50000, '2021-01-15'),
            (2, 'Bob', 'Engineering', 70000, '2020-06-10'),
            (3, 'Charlie', 'Marketing', 60000, '2022-03-20')
        ]

        # Insert sample data into Departments
        departments_data = [
            (1, 'Sales', 'Alice'),
            (2, 'Engineering', 'Bob'),
            (3, 'Marketing', 'Charlie')
        ]

        # Insert data
        cursor.executemany('INSERT INTO Employees VALUES (?,?,?,?,?)', employees_data)
        cursor.executemany('INSERT INTO Departments VALUES (?,?,?)', departments_data)

    conn.commit()
    conn.close()

def get_db_connection():
    """Create a database connection"""
    conn = sqlite3.connect(DATABASE_PATH)
    conn.row_factory = sqlite3.Row
    return conn

def execute_query(query, params=()):
    """Execute a database query"""
    conn = get_db_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(query, params)
        results = cursor.fetchall()
        conn.close()
        return [dict(row) for row in results]
    except sqlite3.Error as e:
        conn.close()
        print(f"Database error: {e}")
        raise e

def parse_query(user_input):
    """Parse natural language query and convert to SQL"""
    user_input = user_input.lower().strip()
    
    # Comprehensive query parsing
    query_patterns = [
        # Employees in a department
        (r'show me all employees in (\w+) department', 
         """SELECT * FROM Employees 
            WHERE LOWER(Department) = LOWER(?)"""),
        
        # Department manager
        (r'who is the manager of (\w+) department', 
         """SELECT Manager FROM Departments 
            WHERE LOWER(Name) = LOWER(?)"""),
        
        # Employees hired after a date
        (r'list all employees hired after (\d{4}-\d{2}-\d{2})', 
         """SELECT * FROM Employees 
            WHERE Hire_Date > ?"""),
        
        # Total salary for a department
        (r'total salary expense for (\w+) department', 
         """SELECT SUM(Salary) as total_salary 
            FROM Employees 
            WHERE LOWER(Department) = LOWER(?)"""),
        
        # List of employees with salary above
        (r'list employees with salary above (\d+)', 
         """SELECT * FROM Employees 
            WHERE Salary > ?""")
    ]
    
    for pattern, query_template in query_patterns:
        match = re.search(pattern, user_input)
        if match:
            return query_template, (match.group(1),)
    
    return None, None

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/query', methods=['POST'])
def handle_query():
    try:
        user_input = request.json.get('query', '')
        query, params = parse_query(user_input)
        
        if query is None:
            return jsonify({
                'error': 'I couldn\'t understand that query. Try these examples:\n' + 
                        '- Show me all employees in Sales department\n' +
                        '- Who is the manager of Engineering department\n' +
                        '- List all employees hired after 2021-01-01\n' +
                        '- Total salary expense for Marketing department'
            })

        results = execute_query(query, params)
        
        if not results:
            return jsonify({
                'message': 'No results found for your query.'
            })

        return jsonify({
            'results': results
        })

    except Exception as e:
        return jsonify({
            'error': f'An error occurred: {str(e)}'
        })

def run_flask_app():
    """Run the Flask application"""
    create_database()  # Ensure database is created before running
    app.run(host='0.0.0.0', port=5000, debug=False)

if __name__ == '__main__':
    # Create database before running
    create_database()
    
    try:
        # Attempt to open browser (works in some environments)
        import webbrowser
        threading.Timer(1.5, lambda: webbrowser.open('http://localhost:5000')).start()
    except ImportError:
        print("Unable to open browser automatically.")
    
    # Run the Flask app
    run_flask_app()

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Database Chat Assistant</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: 800px;
            margin: 0 auto;
            padding: 20px;
            line-height: 1.6;
        }
        .chat-container {
            border: 1px solid #e0e0e0;
            border-radius: 8px;
            padding: 20px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        .input-container {
            display: flex;
            margin-bottom: 15px;
        }
        #query-input {
            flex-grow: 1;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 4px;
        }
        button {
            padding: 10px 15px;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 4px;
            margin-left: 10px;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }
        button:hover {
            background-color: #0056b3;
        }
        #response {
            background-color: #f8f9fa;
            padding: 15px;
            border-radius: 4px;
            white-space: pre-wrap;
            max-height: 300px;
            overflow-y: auto;
        }
        .example-queries {
            margin-top: 20px;
            background-color: #f1f3f5;
            padding: 15px;
            border-radius: 4px;
        }
    </style>
</head>
<body>
    <h1>Database Chat Assistant</h1>
    
    <div class="chat-container">
        <div class="input-container">
            <input type="text" id="query-input" placeholder="Enter your query...">
            <button onclick="sendQuery()">Send Query</button>
        </div>
        <div id="response">Results will appear here...</div>
    </div>

    <div class="example-queries">
        <h3>Example Queries:</h3>
        <ul>
            <li>Show me all employees in Sales department</li>
            <li>Who is the manager of Engineering department</li>
            <li>List all employees hired after 2021-01-01</li>
            <li>Total salary expense for Marketing department</li>
        </ul>
    </div>

    <script>
        async function sendQuery() {
            const input = document.getElementById('query-input');
            const response = document.getElementById('response');
            
            if (!input.value.trim()) {
                response.innerHTML = 'Please enter a query.';
                return;
            }
            
            response.innerHTML = 'Processing...';
            
            try {
                const res = await fetch('/query', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ query: input.value }),
                });
                
                const data = await res.json();
                
                if (data.error) {
                    response.innerHTML = `Error: ${data.error}`;
                } else if (data.message) {
                    response.innerHTML = data.message;
                } else {
                    response.innerHTML = JSON.stringify(data.results, null, 2);
                }
            } catch (error) {
                response.innerHTML = `Error: ${error.message}`;
            }
        }

        // Allow enter key to submit query
        document.getElementById('query-input').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                sendQuery();
            }
        });
    </script>
</body>
</html>

In [ ]:
# Requirements
pip freeze > requirements.txt

# Create Procfile (tells Heroku how to run the app)
web: gunicorn app:app

# Additional files needed
- requirements.txt
- Procfile
- runtime.txt (specify Python version)

# Basic deployment commands
git init
heroku create your-app-name
git add .
git commit -m "Initial deployment"
heroku git:remote -a your-app-name
git push heroku main